In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers import LSTM, Dropout, Dense, Activation
import datetime

In [ ]:
data = pd.read_csv('/content/drive/My Drive/test/185750.KS.csv')
data.head()

Compute Mid Price

In [ ]:
high_prices = data['High'].values
low_prices = data['Low'].values
mid_prices = (high_prices + low_prices) / 2

Create Windows

In [ ]:
seq_len = 50
sequence_length = seq_len + 1

result = []
for index in range(len(mid_prices) - sequence_length):
    result.append(mid_prices[index: index + sequence_length])

Normalize Data

In [ ]:
normalized_data = []
for window in result:
    normalized_window = [((float(p)/float(window[0]))- 1)for p in window] # what is window[0]??
    normalized_data.append(normalized_window)

result = np.array(normalized_data)

# split train set & test set
row = int(round(result.shape[0]*0.9))
train = result[:row, :]
np.random.shuffle(train)

x_train = train[:, :-1]
x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 1))
y_train = train[:, -1]

x_test = result[row:, :-1]
x_test = np.reshape(x_test, (x_test.shape[0], x_test.shape[1], 1))
y_test = result[row:, -1]

x_train.shape, x_test.shape


Build a Model

In [ ]:
model = Sequential()
model.add(LSTM(50, return_sequences=True, input_shape=(50,1))) # 50일 입력
model.add(LSTM(64, return_sequences=False))
model.add(Dense(1, activation='linear'))
model.compile(loss='mse', optimizer='rmsprop')
model.summary()

Training

In [ ]:
model.fit(x_train, y_train, 
          validation_data=(x_test, y_test), 
          batch_size=10,
          epochs=20)

Prediction

In [ ]:
pred = model.predict(x_test)

fig = plt.figure(facecolor='white')
ax = fig.add_subplot(111)
ax.plot(y_test, label='True')
ax.plot(pred, label='Prediction')
ax.legend()
plt.show()
print(pred)